<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **SpaceX  Falcon 9 first stage Landing Prediction**


# Lab 1: Collecting the data


Estimated time needed: **45** minutes


In this capstone, we will predict if the Falcon 9 first stage will land successfully. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars; other providers cost upward of 165 million dollars each, much of the savings is because SpaceX can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against SpaceX for a rocket launch. In this lab, you will collect and make sure the data is in the correct format from an API. The following is an example of a successful and launch.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/crash.gif)


Most unsuccessful landings are planned. Space X performs a controlled landing in the oceans. 


## Objectives


In this lab, you will make a get request to the SpaceX API. You will also do some basic data wrangling and formating. 

- Request to the SpaceX API
- Clean the requested data


----


## Import Libraries and Define Auxiliary Functions


We will import the following libraries into the lab


In [1]:
# Requests allows us to make HTTP requests which we will use to get data from an API
import requests
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Datetime is a library that allows us to represent dates
import datetime

# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)



Below we will define a series of helper functions that will help us use the API to extract information using identification numbers in the launch data.

From the <code>rocket</code> column we would like to learn the booster name.


In [2]:

# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
       if x:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])
        
        

From the <code>launchpad</code> we would like to know the name of the launch site being used, the logitude, and the latitude.


In [4]:


# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])
        


From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to.


In [5]:


# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])
        


From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, wheter the core is reused, wheter legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.


In [6]:


# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])
            


Now let's start requesting rocket launch data from SpaceX API with the following URL:


In [7]:

spacex_url="https://api.spacexdata.com/v4/launches/past"


In [8]:
response = requests.get(spacex_url)

Check the content of the response


In [ ]:
print(response.content)

You should see the response contains massive information about SpaceX launches. Next, let's try to discover some more relevant information for this project.


### Task 1: Request and parse the SpaceX launch data using the GET request


To make the requested JSON results more consistent, we will use the following static response object for this project:


In [10]:


static_json_url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json'



We should see that the request was successfull with the 200 status response code


In [11]:

response=requests.get(static_json_url)


In [12]:
response.status_code

200

Now we decode the response content as a Json using <code>.json()</code> and turn it into a Pandas dataframe using <code>.json_normalize()</code>


In [14]:


# Use json_normalize meethod to convert the json result into a dataframe
import requests
import pandas as pd

# Perform GET request
response = requests.get("https://api.spacexdata.com/v4/launches")
data = response.json()  # Convert response to JSON
df = pd.json_normalize(data)  # Normalize nested JSON


# Check if the column exists (some launches may lack this field)
if 'static_fire_date_utc' in df.columns:
    # Get the first non-null date (handling missing data)
    first_date = df['static_fire_date_utc'].dropna().iloc[0]
    year = pd.to_datetime(first_date).year
    print(f"The year in the first row is: {year}")
else:
    print("Column 'static_fire_date_utc' not found in the DataFrame.")
    
    

The year in the first row is: 2006


Using the dataframe <code>data</code> print the first 5 rows


In [15]:
# Get the head of the dataframe
print(df[['static_fire_date_utc']].head())


       static_fire_date_utc
0  2006-03-17T00:00:00.000Z
1                      None
2                      None
3  2008-09-20T00:00:00.000Z
4                      None


You will notice that a lot of the data are IDs. For example the rocket column has no information about the rocket just an identification number.

We will now use the API again to get information about the launches using the IDs given for each launch. Specifically we will be using columns <code>rocket</code>, <code>payloads</code>, <code>launchpad</code>, and <code>cores</code>.


In [18]:
# Perform GET request and get JSON data
response = requests.get("https://api.spacexdata.com/v4/launches")
data = response.json()  # This returns a list of dictionaries

# Convert the list of dictionaries to a pandas DataFrame
df = pd.json_normalize(data)

# Now take a subset of our dataframe keeping only the features we want
df = df[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# Remove rows with multiple cores (Falcon rockets with 2 extra boosters)
df = df[df['cores'].map(len) == 1]
df = df[df['payloads'].map(len) == 1]

# Extract single values from lists
df['cores'] = df['cores'].map(lambda x: x[0])
df['payloads'] = df['payloads'].map(lambda x: x[0])

# Convert date_utc to datetime and extract date
df['date'] = pd.to_datetime(df['date_utc']).dt.date

# Restrict dates of launches
df = df[df['date'] <= datetime.date(2020, 11, 13)]

* From the <code>rocket</code> we would like to learn the booster name

* From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to

* From the <code>launchpad</code> we would like to know the name of the launch site being used, the longitude, and the latitude.

* **From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, whether the core is reused, whether legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.**

The data from these requests will be stored in lists and will be used to create a new dataframe.


In [ ]:


#Global variables 
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []



These functions will apply the outputs globally to the above variables. Let's take a looks at <code>BoosterVersion</code> variable. Before we apply  <code>getBoosterVersion</code> the list is empty:


In [ ]:
BoosterVersion

Now, let's apply <code> getBoosterVersion</code> function method to get the booster version


In [ ]:


# Call getBoosterVersion
getBoosterVersion(data)



the list has now been update 


In [ ]:
BoosterVersion[0:5]

we can apply the rest of the  functions here:


In [ ]:
# Call getLaunchSite
getLaunchSite(data)

In [ ]:
# Call getPayloadData
getPayloadData(data)

In [ ]:
# Call getCoreData
getCoreData(data)

Finally lets construct our dataset using the data we have obtained. We we combine the columns into a dictionary.


In [ ]:

launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}



Then, we need to create a Pandas data frame from the dictionary launch_dict.


In [19]:


# Create a data from launch_dict


# Global variables to store the collected data
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

# Function to get booster version
def getBoosterVersion(data):
    for x in data['rocket']:
        if x:
            response = requests.get(f"https://api.spacexdata.com/v4/rockets/{x}").json()
            BoosterVersion.append(response['name'])

# Function to get launch site information
def getLaunchSite(data):
    for x in data['launchpad']:
        if x:
            response = requests.get(f"https://api.spacexdata.com/v4/launchpads/{x}").json()
            LaunchSite.append(response['name'])
            Longitude.append(response['longitude'])
            Latitude.append(response['latitude'])

# Function to get payload data
def getPayloadData(data):
    for load in data['payloads']:
        if load:
            response = requests.get(f"https://api.spacexdata.com/v4/payloads/{load}").json()
            PayloadMass.append(response['mass_kg'] if 'mass_kg' in response else None)
            Orbit.append(response['orbit'] if 'orbit' in response else None)

# Function to get core data
def getCoreData(data):
    for core in data['cores']:
        if core['core']:
            response = requests.get(f"https://api.spacexdata.com/v4/cores/{core['core']}").json()
            Block.append(response['block'] if 'block' in response else None)
            ReusedCount.append(response['reuse_count'] if 'reuse_count' in response else None)
            Serial.append(response['serial'] if 'serial' in response else None)
        else:
            Block.append(None)
            ReusedCount.append(None)
            Serial.append(None)
        
        # Add core landing information
        Outcome.append(f"{core['landing_success']} {core['landing_type']}" if 'landing_success' in core else None)
        Flights.append(core['flight'] if 'flight' in core else None)
        GridFins.append(core['gridfins'] if 'gridfins' in core else None)
        Reused.append(core['reused'] if 'reused' in core else None)
        Legs.append(core['legs'] if 'legs' in core else None)
        LandingPad.append(core['landpad'] if 'landpad' in core else None)

# Get the data from API
response = requests.get("https://api.spacexdata.com/v4/launches")
data = response.json()

# Convert to DataFrame and process
df = pd.json_normalize(data)

# Filter and clean the data
df = df[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]
df = df[df['cores'].map(len) == 1]
df = df[df['payloads'].map(len) == 1]
df['cores'] = df['cores'].map(lambda x: x[0])
df['payloads'] = df['payloads'].map(lambda x: x[0])
df['date'] = pd.to_datetime(df['date_utc']).dt.date
df = df[df['date'] <= datetime.date(2020, 11, 13)]

# Call all the functions to populate our global variables
getBoosterVersion(df)
getLaunchSite(df)
getPayloadData(df)
getCoreData(df)

# Create the final dictionary
launch_dict = {
    'FlightNumber': list(df['flight_number']),
    'Date': list(df['date']),
    'BoosterVersion': BoosterVersion,
    'PayloadMass': PayloadMass,
    'Orbit': Orbit,
    'LaunchSite': LaunchSite,
    'Outcome': Outcome,
    'Flights': Flights,
    'GridFins': GridFins,
    'Reused': Reused,
    'Legs': Legs,
    'LandingPad': LandingPad,
    'Block': Block,
    'ReusedCount': ReusedCount,
    'Serial': Serial,
    'Longitude': Longitude,
    'Latitude': Latitude
}

# Create the final DataFrame
spacex_df = pd.DataFrame(launch_dict)

# Display the first few rows of the final DataFrame
spacex_df.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2006-03-24,Falcon 1,20.0,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin1A,167.743129,9.047721
1,2,2007-03-21,Falcon 1,NaN,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin2A,167.743129,9.047721
2,4,2008-09-28,Falcon 1,165.0,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin2C,167.743129,9.047721
3,5,2009-07-13,Falcon 1,200.0,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin3C,167.743129,9.047721
4,6,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857


Show the summary of the dataframe


In [20]:


# Show the head of the dataframe

# Show value counts for categorical columns
print("\nBooster Version counts:")
display(spacex_df['BoosterVersion'].value_counts())

print("\nOrbit counts:")
display(spacex_df['Orbit'].value_counts())



Booster Version counts:


Falcon 9    90
Falcon 1     4
Name: BoosterVersion, dtype: int64


Orbit counts:


GTO      27
ISS      21
VLEO     14
LEO      11
PO        9
SSO       5
MEO       3
ES-L1     1
HEO       1
SO        1
GEO       1
Name: Orbit, dtype: int64

### Task 2: Filter the dataframe to only include `Falcon 9` launches


Finally we will remove the Falcon 1 launches keeping only the Falcon 9 launches. Filter the data dataframe using the <code>BoosterVersion</code> column to only keep the Falcon 9 launches. Save the filtered data to a new dataframe called <code>data_falcon9</code>.


In [21]:


# Hint data['BoosterVersion']!='Falcon 1'

# Filter to keep only Falcon 9 launches
data_falcon9 = spacex_df[spacex_df['BoosterVersion'] == 'Falcon 9']

# Verify the filtering by checking the unique booster versions
print("Unique booster versions after filtering:")
print(data_falcon9['BoosterVersion'].unique())

# Show the shape of the new dataframe
print(f"\nOriginal dataframe shape: {spacex_df.shape}")
print(f"Filtered dataframe shape: {data_falcon9.shape}")

# Show the first 5 rows of the filtered dataframe
print("\nFirst 5 rows of Falcon 9 launches:")
display(data_falcon9.head())


Unique booster versions after filtering:
['Falcon 9']

Original dataframe shape: (94, 17)
Filtered dataframe shape: (90, 17)

First 5 rows of Falcon 9 launches:


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
4,6,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857
5,8,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0005,-80.577366,28.561857
6,10,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0007,-80.577366,28.561857
7,11,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,0,B1003,-120.610829,34.632093
8,12,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1004,-80.577366,28.561857


Now that we have removed some values we should reset the FlgihtNumber column


In [23]:

#data_falcon9.loc[:,'FlightNumber'] = list(range(1, data_falcon9.shape[0]+1))
#data_falcon9

# Filter to keep only Falcon 9 launches
data_falcon9 = spacex_df[spacex_df['BoosterVersion'].str.contains('Falcon 9')].copy()

# Reset the FlightNumber column to be sequential starting from 1
data_falcon9.loc[:, 'FlightNumber'] = list(range(1, len(data_falcon9) + 1))

# Verify the changes
print("Flight numbers after resetting:")
print(data_falcon9[['FlightNumber', 'BoosterVersion']].head())

print("\nFlight numbers before resetting (original data):")
print(spacex_df[['FlightNumber', 'BoosterVersion']].head())



Flight numbers after resetting:
   FlightNumber BoosterVersion
4             1       Falcon 9
5             2       Falcon 9
6             3       Falcon 9
7             4       Falcon 9
8             5       Falcon 9

Flight numbers before resetting (original data):
   FlightNumber BoosterVersion
0             1       Falcon 1
1             2       Falcon 1
2             4       Falcon 1
3             5       Falcon 1
4             6       Falcon 9


## Data Wrangling


We can see below that some of the rows are missing values in our dataset.


In [33]:
data_falcon9.isnull().sum()

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        5
Orbit              0
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
Success            0
dtype: int64

Before we can continue we must deal with these missing values. The <code>LandingPad</code> column will retain None values to represent when landing pads were not used.


### Task 3: Dealing with Missing Values


Calculate below the mean for the <code>PayloadMass</code> using the <code>.mean()</code>. Then use the mean and the <code>.replace()</code> function to replace `np.nan` values in the data with the mean you calculated.


In [34]:
# Step 1: Check current missing values
print("Missing values before handling:")
print(data_falcon9.isnull().sum())

# Step 2: Calculate mean of PayloadMass (excluding NaN values)
payload_mean = data_falcon9['PayloadMass'].mean()
print(f"\nMean Payload Mass: {payload_mean:.2f} kg")

# Step 3: Replace NaN values with the mean
data_falcon9['PayloadMass'] = data_falcon9['PayloadMass'].fillna(payload_mean)

# Step 4: Verify NaN values were replaced
print("\nMissing values after handling:")
print(data_falcon9.isnull().sum())

# Step 5: Export to CSV (LandingPad will retain None values)
data_falcon9.to_csv('dataset_part_1.csv', index=False)
print("\nData saved to 'dataset_part_1.csv'")

Missing values before handling:
FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        5
Orbit              0
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
Success            0
dtype: int64

Mean Payload Mass: 6123.55 kg

Missing values after handling:
FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        0
Orbit              0
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
Success            0
dtype: int64

Data saved to 'dataset_part_1.csv'


You should see the number of missing values of the <code>PayLoadMass</code> change to zero.


Now we should have no missing values in our dataset except for in <code>LandingPad</code>.


We can now export it to a <b>CSV</b> for the next section,but to make the answers consistent, in the next lab we will provide data in a pre-selected date range. 


<code>data_falcon9.to_csv('dataset_part_1.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD. 


<!--## Change Log
-->


<!--

|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2020-09-20|1.1|Joseph|get result each time you run|
|2020-09-20|1.1|Azim |Created Part 1 Lab using SpaceX API|
|2020-09-20|1.0|Joseph |Modified Multiple Areas|
-->


Copyright ©IBM Corporation. All rights reserved.
